# <span style="color: blue;">Helper Methods :-</span>

### <span style="color: green;">This file has the helper methods needed for building and running a model</span>

In [47]:
from IPython.display import HTML
HTML('''<script>
    code_show_err=false; 
    function code_toggle_err() {
     if (code_show_err){
     $('div.output_stderr').hide();
     } else {
     $('div.output_stderr').show();
     }
     code_show_err = !code_show_err
    } 
    $( document ).ready(code_toggle_err);
    </script>
    To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [48]:

import pandas as pd
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
import tensorflow as tf
import spacy

In [49]:
#Twitter feed related imports

# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
from datetime import datetime, timedelta
import dateutil.parser
import unicodedata

import pytz

from pyrfc3339 import generate, parse

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from imblearn.over_sampling import SMOTE
from transformers import InputExample, InputFeatures
from sklearn.base import TransformerMixin
from ipynb.fs.full.TwitterFeed import get_tweets_df

In [51]:
tweets = pd.read_csv('data/train.csv')
tweet=tweets[['text','target']]

# <span style="color: green;">Method to print the bar graph based on data :-</span>

In [52]:
def bar_plot(data):
    data.value_counts().sort_values().plot(kind = 'bar')

# <span style="color: green;">Method to apply text with Stemmer applied to it :-</span>

In [53]:
def stemmer(text):
    stemm = PorterStemmer()
    return ' '.join([stemm.stem(w) for w in word_tokenize(text)])

# <span style="color: green;">Method to apply text with Lemmetizer applied to it :-</span>

In [54]:
def lemmetizer(text):
    lemmetizer=WordNetLemmatizer()
    return ' '.join([lemmetizer.lemmatize(w) for w in word_tokenize(text)])

# <span style="color: green;">Method to balance an imbalanced class:-</span>

In [55]:
def class_balancer(X,y):
    sampler = SMOTE(random_state=42,k_neighbors=2)
    X1, y1 = sampler.fit_resample(X, y)
    return X1,y1

In [56]:
tweet['stemmer']=tweet.text.apply(stemmer)
tweet['lemmetizer']=tweet.text.apply(lemmetizer)
tweet

/var/folders/5f/5g9q6dvj03n7h1pj6bhdjfkw0000gq/T/ipykernel_9424/2266130295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['stemmer']=tweet.text.apply(stemmer)
/var/folders/5f/5g9q6dvj03n7h1pj6bhdjfkw0000gq/T/ipykernel_9424/2266130295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['lemmetizer']=tweet.text.apply(lemmetizer)


,text,target,stemmer,lemmetizer
0,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of thi # earthquak may...,Our Deeds are the Reason of this # earthquake ...
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask . canada,Forest fire near La Ronge Sask . Canada
2,All residents asked to 'shelter in place' are ...,1,all resid ask to 'shelter in place ' are be no...,All resident asked to 'shelter in place ' are ...
3,"13,000 people receive #wildfires evacuation or...",1,"13,000 peopl receiv # wildfir evacu order in c...","13,000 people receive # wildfire evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...,1,just got sent thi photo from rubi # alaska as ...,Just got sent this photo from Ruby # Alaska a ...
...,...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,1,two giant crane hold a bridg collaps into near...,Two giant crane holding a bridge collapse into...
7609,@aria_ahrary @TheTawniest The out of control w...,1,@ aria_ahrari @ thetawniest the out of control...,@ aria_ahrary @ TheTawniest The out of control...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m1.94 [ 01:04 utc ] ? 5km s of volcano hawaii ...,M1.94 [ 01:04 UTC ] ? 5km S of Volcano Hawaii ...
7611,Police investigating after an e-bike collided ...,1,polic investig after an e-bik collid with a ca...,Police investigating after an e-bike collided ...


In [57]:
X=tweet.drop('target',axis=1)
y=tweet['target']


# <span style="color: green;">Method to return the train and test data with stemmer applied text :-</span>

In [58]:
def stemm_data():
    X_train,X_test,y_train,y_test=train_test_split(X['stemmer'],y,test_size=0.25,random_state=42)
    return X_train,X_test,y_train,y_test

# <span style="color: green;">Method to return the train and test data with stemmer applied text :-</span>

In [59]:
def lemm_data():
    X_train,X_test,y_train,y_test=train_test_split(X['lemmetizer'],y,test_size=0.25,random_state=42)
    return X_train,X_test,y_train,y_test

# <span style="color: green;">Helper Method to return the data loaded :-</span>

In [60]:
def get_data():
    return X,y

# <span style="color: green;">The data frame to collect the metrics data :-</span>

In [61]:
comparison= pd.DataFrame({'model': [], 
             'best_params': [],
             'best_score': [],
             'best_time':[],
             'accuracy_score':[],
             'roc_auc_score':[]})
comparison=comparison.astype(str)
print(comparison.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model           0 non-null      object
 1   best_params     0 non-null      object
 2   best_score      0 non-null      object
 3   best_time       0 non-null      object
 4   accuracy_score  0 non-null      object
 5   roc_auc_score   0 non-null      object
dtypes: object(6)
memory usage: 124.0+ bytes
None


# <span style="color: green;">Helper Method to fit the model and record the fit time :-</span>

In [62]:
def model_execute(model,X,y):
    start=time.time()
    model.fit(X,y)
    end=time.time()
    return end-start

# <span style="color: green;">Helper Method to return the train and test data with stemmer applied text :-</span>

In [63]:
def print_model_data(name,time,model,X_train,X_test,y_train,y_test,is_grid,title='Confusion Matrix',cmap=plt.cm.Blues):
    score = model.score(X_test,y_test)
    model_param = model.get_params()
    pred=model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    auc = roc_auc_score(y_test,pred)    
    conf = metrics.ConfusionMatrixDisplay.from_predictions(y_test, pred)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicated label')
    if(is_grid==True):
        best_params=model.best_params_
        comparison.loc[len(comparison.index)] = [name,best_params,score,time,accuracy,auc]
    else:
        comparison.loc[len(comparison.index)] = [name,model_param,score,time,accuracy,auc]
    return comparison

# <span style="color: green;">Helper Method to clean up the data for class balancing :-</span>

In [64]:
def data_cleanup_for_balancer_stemmer(X):
    vectorizer = CountVectorizer()

    vectorizer.fit(X['stemmer'].values.ravel())

    X_t=vectorizer.transform(X['stemmer'].values.ravel())

    X_train,X_test,y_train,y_test=train_test_split(X_t,y,test_size=0.3,random_state=42)
    
    return X_train,X_test,y_train,y_test

# <span style="color: green;">Helper Method to clean up the data for class balancing :-</span>

In [65]:
def data_cleanup_for_balancer_lemmetizer(X):
    vectorizer = CountVectorizer()

    vectorizer.fit(X['lemmetizer'].values.ravel())

    X_t=vectorizer.transform(X['lemmetizer'].values.ravel())

    X_train,X_test,y_train,y_test=train_test_split(X_t,y,test_size=0.3,random_state=42)
    
    return X_train,X_test,y_train,y_test

# <span style="color: green;">Method to do the text cleaning:-</span>

In [66]:
#reference https://michael-fuchs-python.netlify.app/2021/05/22/nlp-text-pre-processing-i-text-cleaning/#text-cleaning

def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

# <span style="color: green;">Helper Method to work with Tensor Flow :-</span>

In [67]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
    return train_InputExamples, validation_InputExamples

    train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  


# <span style="color: green;">Helper method to work with Tensor flow :-</span>

In [68]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

# <span style="color: green;">Helper method to get live tweet data for the last 15 mins capped to 15 record:-</span>

In [71]:
def get_live_tweets():
    return get_tweets_df()